In [1]:
import pandas as pd
import requests
import json
import pymysql
import getpass

In [2]:
apikey = getpass.getpass()

········


In [16]:
headers = {"Content-Type": "application/json"}
auth = ("apikey", apikey)
url = "https://api.eu-gb.language-translator.watson.cloud.ibm.com/instances/d39ad4d8-67ff-4e11-9d6b-72ac4ae95d14/v3/translate?version=2018-05-01"

In [17]:
data = {"text": ["Hello"],
        "model_id": "en-es"}

put 'data' into a json object

In [18]:
json.dumps(data)

'{"text": ["Hello"], "model_id": "en-es"}'

In [19]:
test = requests.post(auth=auth, headers=headers, url=url, data=json.dumps(data))
test

<Response [200]>

In [21]:
test.text

'{\n  "translations" : [ {\n    "translation" : "Hola"\n  } ],\n  "word_count" : 1,\n  "character_count" : 5\n}'

In [22]:
test.content

b'{\n  "translations" : [ {\n    "translation" : "Hola"\n  } ],\n  "word_count" : 1,\n  "character_count" : 5\n}'

In [23]:
test.json()

{'translations': [{'translation': 'Hola'}],
 'word_count': 1,
 'character_count': 5}

In [40]:
def translate_en_es(word):
    to_translate = {"text": [word],
                    "model_id": "en-es"}
    res = requests.post(auth=auth, headers=headers, url=url, data=json.dumps(to_translate))
    (transl_dict,) = res.json()["translations"]
    return transl_dict["translation"]

In [41]:
translate_en_es("always")

'siempre'

In [42]:
translate_en_es("damn")

'Maldita sea.'

## Translate product category names from olist dataset from portuguese to english

In [43]:
pw = getpass.getpass()

········


In [44]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       passwd=pw,
                       db="olist")

In [45]:
query = """
SELECT DISTINCT
    product_category_name
FROM products
WHERE product_category_name IS NOT NULL;
"""

In [46]:
product_categories = pd.read_sql(query, conn)

In [47]:
product_categories.head()

,product_category_name
0,perfumaria
1,artes
2,esporte_lazer
3,bebes
4,utilidades_domesticas


Get a new column with english translations:

1. replace underscores with whitespaces
2. get the category names out of df into a list of strings
3. send translation request
4. extract translations from request response
5. create new column with translations

In [50]:
# 1. replace underscores with whitespaces
product_categories = product_categories.assign(product_category_string=(product_categories["product_category_name"]
                                                                        .str
                                                                        .replace("_", " ")))

In [53]:
# 2. get the category names out of df into a list of strings
product_category_list = product_categories["product_category_string"].to_list()

In [54]:
# 3. send translation request
to_translate = {"text": product_category_list,
                "model_id": "pt-en"}

r = requests.post(auth=auth, headers=headers, url=url, data=json.dumps(to_translate))

In [58]:
# 4. extract translations from request response
pd.DataFrame(r.json()["translations"])

,translation
0,perfumery
1,arts
2,sport leisure
3,babies
4,domestical utilities
...,...
68,house comfort 2
69,portable cooking and food preparers
70,safe and servic
71,movable collchao and upholstery


In [61]:
# 5. create new column with translations
product_categories = product_categories.assign(product_category_name_en=pd.DataFrame(r.json()["translations"]))

In [63]:
product_categories

,product_category_name,product_category_string,product_category_name_en
0,perfumaria,perfumaria,perfumery
1,artes,artes,arts
2,esporte_lazer,esporte lazer,sport leisure
3,bebes,bebes,babies
4,utilidades_domesticas,utilidades domesticas,domestical utilities
...,...,...,...
68,casa_conforto_2,casa conforto 2,house comfort 2
69,portateis_cozinha_e_preparadores_de_alimentos,portateis cozinha e preparadores de alimentos,portable cooking and food preparers
70,seguros_e_servicos,seguros e servicos,safe and servic
71,moveis_colchao_e_estofado,moveis colchao e estofado,movable collchao and upholstery
